# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load the datasdet dair-air/emotion and explore the data

In [1]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [2]:
import random

# print some random featues and the labels
print("Features:")
indices = random.sample(range(len(ds["train"])), 10)
for i in indices:
    print("{} : {}".format(ds["train"]['text'][i], ds["train"]['label'][i]))

print("\nLabels: {}".format(ds["train"].features["label"].names))

Features:
i want to feel like i m reading something worthwhile : 1
i have a feeling this month is going to have some damn cool things in store : 1
im happy to say im feeling so much more creative than i have in a long time : 1
i don t want to go all very special episode of blossom on you but i am feeling a little melancholy about the final episode of rock : 0
i feel heartbroken again i feel dead inside lost angry at myself : 0
i understand the feeling so i wouldnt be shocked : 5
i inquire incheswhyinches are people relocating droves about what they feel is security in precious metal : 1
i feel wonderful monroe said upon the launch of her company im incorporated : 1
i stared up at him amazed by the feeling and as equally amazed that nothing else was happening : 5
im feeling today as about how i liked the books when i read them if i made this list tomorrow it would be different : 2

Labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [3]:
# create data structures for further processing

# names of the splits
splits=list(ds.keys())
# number of classes
num_classes=len(ds["train"].features["label"].names)

# Dictionairies to translate between label string and label number
id2label = dict(zip(range(num_classes), ds['train'].features['label'].names))
label2id = dict(zip(ds['train'].features['label'].names, range(num_classes)))
print(id2label)
print(label2id)

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}


Create a base model with added padding token

In [4]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.accelerator.current_accelerator().type if hasattr(torch, "accelerator") else "cuda"

# Create a base model variant with classification head
def create_base_model(model_id):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id, 
        num_labels=num_classes,
        id2label=id2label,
        label2id=label2id,
        device_map=device)
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

    return model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification

# Use GPT-2 as a small base model
model_id = "openai-community/gpt2"
model = create_base_model(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add tokens to the dataset
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

for param in model.base_model.parameters():
    param.requires_grad = False

# Add the padding token which is missing in GPT-2
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import os

temp_path = "/tmp"
save_path = "./data"

model_name = "gpt2_classification"
checkpoint_dir = os.path.join(temp_path, model_name)
save_dir_base = os.path.join(save_path, model_name)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=checkpoint_dir,
        learning_rate=2e-3,
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        num_train_epochs=4,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.311079,0.504000
2,No log,1.245872,0.537500
3,No log,1.229976,0.542500
4,1.311100,1.223381,0.546500


TrainOutput(global_step=640, training_loss=1.2824213027954101, metrics={'train_runtime': 59.2461, 'train_samples_per_second': 1080.24, 'train_steps_per_second': 10.802, 'total_flos': 1848843351244800.0, 'train_loss': 1.2824213027954101, 'epoch': 4.0})

In [7]:
# Evaluate the model
original_performance=trainer.evaluate()
print(original_performance)

model.save_pretrained(save_dir_base)

{'eval_loss': 1.2233811616897583, 'eval_accuracy': 0.5465, 'eval_runtime': 1.3104, 'eval_samples_per_second': 1526.301, 'eval_steps_per_second': 15.263, 'epoch': 4.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [34]:
from peft import LoraConfig, TaskType, get_peft_model

# Use Lora for PEFT
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    task_type=TaskType.TOKEN_CLS,
    fan_in_fan_out=True,
)

# create PEFT model, use a fresh pretrained base model
model_lora = get_peft_model(create_base_model(model_id), peft_config)
model_lora.print_trainable_parameters()

model_name = "gpt2_classification_lora"
checkpoint_dir = os.path.join(temp_path, model_name)
save_dir = os.path.join(save_path, model_name)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 594,432 || all params: 125,038,848 || trainable%: 0.4754


In [35]:
trainer_lora = Trainer(
    model=model_lora,
    args=TrainingArguments(
        output_dir=checkpoint_dir,
        learning_rate=2e-3,
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        num_train_epochs=4,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.221460,0.913500
2,No log,0.164348,0.929500
3,No log,0.148877,0.923500
4,0.364900,0.127431,0.925000


TrainOutput(global_step=640, training_loss=0.3104581475257874, metrics={'train_runtime': 70.4535, 'train_samples_per_second': 908.401, 'train_steps_per_second': 9.084, 'total_flos': 1861763687424000.0, 'train_loss': 0.3104581475257874, 'epoch': 4.0})

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [36]:
# Saving the model
model_lora.save_pretrained(save_dir)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [37]:
from peft import PeftModelForTokenClassification

model_id = "openai-community/gpt2"
model_base = create_base_model(model_id)

model_loaded = PeftModelForTokenClassification.from_pretrained(model_base, save_dir)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
trainer_evaluate = Trainer(
    model=model_loaded,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_lora_evaluate",
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        do_train=False,
        do_eval=True,
    ),
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

fine_tuned_performance=trainer_evaluate.evaluate()

In [39]:

print("Original Model:  ", original_performance)
print("Fine-Tuned Model:", fine_tuned_performance)

print("Original Model accurcy:   ", original_performance['eval_accuracy'])
print("Fine-Tuned Model accurcy: ", fine_tuned_performance['eval_accuracy'])

Original Model:   {'eval_loss': 1.2233811616897583, 'eval_accuracy': 0.5465, 'eval_runtime': 1.3104, 'eval_samples_per_second': 1526.301, 'eval_steps_per_second': 15.263, 'epoch': 4.0}
Fine-Tuned Model: {'eval_loss': 0.12743067741394043, 'eval_model_preparation_time': 0.0019, 'eval_accuracy': 0.925, 'eval_runtime': 0.8968, 'eval_samples_per_second': 2230.054, 'eval_steps_per_second': 22.301}
Original Model accurcy:    0.5465
Fine-Tuned Model accurcy:  0.925


### Use different Quantization: QLoRA

In [ ]:
from transformers import BitsAndBytesConfig

torch.cuda.empty_cache()

model_id = "openai-community/gpt2"
temp_path = "/tmp"
save_path = "./data"

model_name = "gpt2_classification_4bit_lora"
checkpoint_dir = os.path.join(temp_path, model_name)
save_dir_base = os.path.join(save_path, model_name)

model_id = "openai-community/gpt2"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model4b = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    num_labels=num_classes,
    id2label=id2label,
    label2id=label2id,
    torch_dtype="auto")

model4b.config.pad_token_id = model.config.eos_token_id

for param in model4b.base_model.parameters():
    param.requires_grad = False

# peft model
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    task_type=TaskType.TOKEN_CLS,
    fan_in_fan_out=True,
)

model4bl = get_peft_model(model4b, peft_config)
model4bl.print_trainable_parameters()

trainer = Trainer(
    model=model4bl,
    args=TrainingArguments(
        output_dir=checkpoint_dir,
        learning_rate=2e-3,
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        num_train_epochs=5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        fp16=True
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
validation_lora_q4 = trainer.evaluate()
#model4bl.save_pretrained(save_dire)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 594,432 || all params: 125,038,848 || trainable%: 0.4754


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.250119,0.909000
2,No log,0.159906,0.932500
3,No log,0.133373,0.929000
4,0.391100,0.123560,0.932500
5,0.391100,0.119329,0.930000


In [19]:
print("Original Model accuracy:         ", original_performance['eval_accuracy'])
print("Fine-Tuned Model accuracy:       ", fine_tuned_performance['eval_accuracy'])
print("Fine-Tuned Model 4 bit accuracy: ", validation_lora_q4['eval_accuracy'])

Original Model accuracy:          0.543
Fine-Tuned Model accuracy:        0.9285
Fine-Tuned Model 4 bit accuracy:  0.9285


### Experiment with different LoRA parameters

In [20]:
from peft import PeftModelForTokenClassification, LoraConfig, TaskType, get_peft_model
import pandas as pd

torch.cuda.empty_cache()


def create_lora_config(r, lora_alpha, lora_dropout):
    peft_config = LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        task_type=TaskType.TOKEN_CLS,
        fan_in_fan_out=True,
    )

    return peft_config

def create_lora_model(peft_config):
    model_base = create_base_model(model_id)
    model_lora = get_peft_model(model_base, peft_config)

    return model_lora
    
def create_trainer(model, learning_rate, weight_decay):
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir='/tmp',
            per_device_train_batch_size=50,
            per_device_eval_batch_size=50,
            num_train_epochs=4,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
        ),
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["test"],
        processing_class=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    return trainer

def evaluate_model(model):
    eval = model.evaluate()

results = []

In [21]:
for r in [8, 4, 2]:
    dropout = 0.1
    learning_rate = 2e-3
    weight_decay = 0.01
    alpha = 2 * r
    config = create_lora_config(r, alpha, dropout)
    model = create_lora_model(config)
    trainer = create_trainer(model, learning_rate, weight_decay)

    print("Start training a model with R={}, alpha={} droptout={}".format(r, alpha, dropout))

    trainer.train()
    eval = trainer.evaluate()

    accuracy = eval['eval_accuracy']
    results.append({'r': r, 'alpha': alpha, 'dropout': dropout, 'learning_rate':learning_rate, 'weight_decay': weight_decay, 'accuracy': accuracy})
    df_results = pd.DataFrame(results)
    print(df_results)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.203967,0.917500
2,0.481400,0.164355,0.929000
3,0.481400,0.152113,0.929500
4,0.165000,0.131185,0.931000


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1          0.002          0.01     0.931


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1          0.002          0.01     0.931
1  4      8      0.1          0.002          0.01     0.928


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=2, alpha=4 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.243588,0.911500
2,0.528500,0.180413,0.922500
3,0.528500,0.171336,0.922500
4,0.196700,0.148653,0.921500


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1          0.002          0.01    0.9310
1  4      8      0.1          0.002          0.01    0.9280
2  2      4      0.1          0.002          0.01    0.9215


In [23]:
for learning_rate in [2e-4, 2e-3, 2e-2]:
    dropout = 0.1
    r = 8
    weight_decay = 0.01
    alpha = 2 * r
    config = create_lora_config(r, alpha, dropout)
    model = create_lora_model(config)
    trainer = create_trainer(model, learning_rate, weight_decay)

    print("Start training a model with R={}, alpha={} droptout={}".format(r, alpha, dropout))

    trainer.train()
    eval = trainer.evaluate()

    accuracy = eval['eval_accuracy']
    results.append({'r': r, 'alpha': alpha, 'dropout': dropout, 'learning_rate':learning_rate, 'weight_decay': weight_decay, 'accuracy': accuracy})
    df_results = pd.DataFrame(results)
    print(df_results)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.514827,0.806500
2,1.004500,0.336314,0.875500
3,1.004500,0.292412,0.893500
4,0.397000,0.268815,0.899000


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1         0.0020          0.01    0.9310
1  4      8      0.1         0.0020          0.01    0.9280
2  2      4      0.1         0.0020          0.01    0.9215
3  8     16      0.1         0.0002          0.01    0.8990


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1         0.0020          0.01    0.9310
1  4      8      0.1         0.0020          0.01    0.9280
2  2      4      0.1         0.0020          0.01    0.9215
3  8     16      0.1         0.0002          0.01    0.8990
4  8     16      0.1         0.0020          0.01    0.9305


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.046261,0.295500
2,2.357200,2.046261,0.295500
3,2.357200,2.046261,0.295500
4,2.089400,2.046261,0.295500


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16      0.1         0.0020          0.01    0.9310
1  4      8      0.1         0.0020          0.01    0.9280
2  2      4      0.1         0.0020          0.01    0.9215
3  8     16      0.1         0.0002          0.01    0.8990
4  8     16      0.1         0.0020          0.01    0.9305
5  8     16      0.1         0.0200          0.01    0.2955


In [24]:
for dropout in [0.01, 0.1, 0.5]:
    learning_rate = 2e-3
    r = 8
    weight_decay = 0.01
    alpha = 2 * r
    config = create_lora_config(r, alpha, dropout)
    model = create_lora_model(config)
    trainer = create_trainer(model, learning_rate, weight_decay)

    print("Start training a model with R={}, alpha={} droptout={}".format(r, alpha, dropout))

    trainer.train()
    eval = trainer.evaluate()

    accuracy = eval['eval_accuracy']
    results.append({'r': r, 'alpha': alpha, 'dropout': dropout, 'learning_rate':learning_rate, 'weight_decay': weight_decay, 'accuracy': accuracy})
    df_results = pd.DataFrame(results)
    print(df_results)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.01


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.222568,0.913500
2,0.470900,0.146687,0.933500
3,0.470900,0.136879,0.929000
4,0.156000,0.117551,0.934500


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16     0.10         0.0020          0.01    0.9310
1  4      8     0.10         0.0020          0.01    0.9280
2  2      4     0.10         0.0020          0.01    0.9215
3  8     16     0.10         0.0002          0.01    0.8990
4  8     16     0.10         0.0020          0.01    0.9305
5  8     16     0.10         0.0200          0.01    0.2955
6  8     16     0.01         0.0020          0.01    0.9345


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16     0.10         0.0020          0.01    0.9310
1  4      8     0.10         0.0020          0.01    0.9280
2  2      4     0.10         0.0020          0.01    0.9215
3  8     16     0.10         0.0002          0.01    0.8990
4  8     16     0.10         0.0020          0.01    0.9305
5  8     16     0.10         0.0200          0.01    0.2955
6  8     16     0.01         0.0020          0.01    0.9345
7  8     16     0.10         0.0020          0.01    0.9305


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.5


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.224562,0.917000
2,0.526600,0.179373,0.924500
3,0.526600,0.161703,0.924000
4,0.203900,0.133277,0.927000


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16     0.10         0.0020          0.01    0.9310
1  4      8     0.10         0.0020          0.01    0.9280
2  2      4     0.10         0.0020          0.01    0.9215
3  8     16     0.10         0.0002          0.01    0.8990
4  8     16     0.10         0.0020          0.01    0.9305
5  8     16     0.10         0.0200          0.01    0.2955
6  8     16     0.01         0.0020          0.01    0.9345
7  8     16     0.10         0.0020          0.01    0.9305
8  8     16     0.50         0.0020          0.01    0.9270


In [25]:
for weight_decay in [0.001, 0.01, 0.1]:
    learning_rate = 2e-3
    dropout = 0.1
    r = 8
    alpha = 2 * r
    config = create_lora_config(r, alpha, dropout)
    model = create_lora_model(config)
    trainer = create_trainer(model, learning_rate, weight_decay)

    print("Start training a model with R={}, alpha={} droptout={}".format(r, alpha, dropout))

    trainer.train()
    eval = trainer.evaluate()

    accuracy = eval['eval_accuracy']
    results.append({'r': r, 'alpha': alpha, 'dropout': dropout, 'learning_rate':learning_rate, 'weight_decay': weight_decay, 'accuracy': accuracy})
    df_results = pd.DataFrame(results)
    print(df_results)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.232432,0.912500
2,0.490000,0.167103,0.927500
3,0.490000,0.159356,0.926000
4,0.161000,0.125772,0.929500


   r  alpha  dropout  learning_rate  weight_decay  accuracy
0  8     16     0.10         0.0020         0.010    0.9310
1  4      8     0.10         0.0020         0.010    0.9280
2  2      4     0.10         0.0020         0.010    0.9215
3  8     16     0.10         0.0002         0.010    0.8990
4  8     16     0.10         0.0020         0.010    0.9305
5  8     16     0.10         0.0200         0.010    0.2955
6  8     16     0.01         0.0020         0.010    0.9345
7  8     16     0.10         0.0020         0.010    0.9305
8  8     16     0.50         0.0020         0.010    0.9270
9  8     16     0.10         0.0020         0.001    0.9295


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

    r  alpha  dropout  learning_rate  weight_decay  accuracy
0   8     16     0.10         0.0020         0.010    0.9310
1   4      8     0.10         0.0020         0.010    0.9280
2   2      4     0.10         0.0020         0.010    0.9215
3   8     16     0.10         0.0002         0.010    0.8990
4   8     16     0.10         0.0020         0.010    0.9305
5   8     16     0.10         0.0200         0.010    0.2955
6   8     16     0.01         0.0020         0.010    0.9345
7   8     16     0.10         0.0020         0.010    0.9305
8   8     16     0.50         0.0020         0.010    0.9270
9   8     16     0.10         0.0020         0.001    0.9295
10  8     16     0.10         0.0020         0.010    0.9305


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training a model with R=8, alpha=16 droptout=0.1


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.229873,0.915500
2,0.488500,0.194819,0.926000
3,0.488500,0.143995,0.929500
4,0.165700,0.125075,0.935000


    r  alpha  dropout  learning_rate  weight_decay  accuracy
0   8     16     0.10         0.0020         0.010    0.9310
1   4      8     0.10         0.0020         0.010    0.9280
2   2      4     0.10         0.0020         0.010    0.9215
3   8     16     0.10         0.0002         0.010    0.8990
4   8     16     0.10         0.0020         0.010    0.9305
5   8     16     0.10         0.0200         0.010    0.2955
6   8     16     0.01         0.0020         0.010    0.9345
7   8     16     0.10         0.0020         0.010    0.9305
8   8     16     0.50         0.0020         0.010    0.9270
9   8     16     0.10         0.0020         0.001    0.9295
10  8     16     0.10         0.0020         0.010    0.9305
11  8     16     0.10         0.0020         0.100    0.9350
